# QCoDeS tutorial 
Basic overview of QCoDeS

## Typical QCodes workflow 
1. Start up an interactive python session (e.g. using jupyter) 
2. import desired modules 
3. instantiate required instruments 
4. experiment! 

In [ ]:
import matplotlib.pyplot as plt
from pprint import pprint
import time
import numpy as np
import qcodes as qc

%matplotlib notebook

In [ ]:
qc.show_subprocess_widget()

In [ ]:
# Ensure that multiprocessing is turned on
qc.config['core']['legacy_mp'] = True
qc.config.save_to_home()

## Instantiates all the instruments needed for the demo

In [ ]:
# spawn doesn't like function or class definitions in the interpreter
# session - had to move them to a file.
from toymodel import AModel, MockGates, MockSource, MockMeter, AverageGetter, AverageAndRaw

# now create this "experiment", note that all these are instruments 
model = AModel()
gates = MockGates('gates', model=model, server_name='')
source = MockSource('source', model=model, server_name='')
meter = MockMeter('meter', model=model, server_name='')

# The station is a container for all instruments that makes it easy 
# to log meta-data
station = qc.Station(gates, source, meter)

# it's nice to have the key parameters be part of the global namespace
# that way they're objects that we can easily set, get, and slice
c0, c1, c2, vsd = gates.chan0, gates.chan1, gates.chan2, source.amplitude

### The location provider can be set globally 

In [ ]:
# dm = qc.data.manager.get_data_manager()
loc_provider = qc.data.location.FormatLocation(fmt='data/{date}/#{counter}_{name}_{time}')
qc.data.data_set.DataSet.location_provider=loc_provider

# Running an experiment 

### Defining the `Loop` and actions

Before you run a measurement loop you do two things:
1. You describe what parameter(s) to vary and how. This is the creation of a `Loop` object: `loop = Loop(sweep_values, ...)`
2. You describe what to do at each step in the loop. This is `loop.each(*actions)` which converts the `Loop` object into an `ActiveLoop` object. Actions can be:
  - measurements (any object with a `.get` method will be interpreted as a measurement)
  - `Task`: some callable (which can have arguments with it) to be executed each time through the loop. Does not generate data.
  - `Wait`: a specialized `Task` just to wait a certain time.
  - `BreakIf`: some condition that, if it returns truthy, breaks (this level of) the loop
  - Another `ActiveLoop` to nest inside the outer one.

For more details, see issue #232 docs: Write bigger picture

In [ ]:
# Notice that one can use an explicit location and `overwrite=True` here so that
# running this notebook over and over won't result in extra files.
# If you leave these out, you get a new timestamped DataSet each time.

loop = qc.Loop(
    c0.sweep(0,20,0.1), delay=0.001).each(
        meter.amplitude)
data = loop.run(name='my_first_measurement')

In [ ]:
qc.MatPlot(data.meter_amplitude)

## Output of the loop
Notice the **"DataSet"**.  
A loop returns a dataset.  
The representation of the dataset shows what arrays it contains and where it is saved.   
The dataset initially starts out empty (filled with NAN's) and get's filled while the Loop get's executed. 

Once the measurement is done, take a look at the file in finder/explorer (the dataset.location should give you the relative path). 
Note also the snapshot that captures the settings of all instruments at the start of the Loop. 
This metadata is also accesible from the dataset and captures a snapshot of each instrument listed in the station. 

In [ ]:
data.metadata

## Plotting the loop  II

QCodes supports both matplotlib inline plotting and pyqtgraph for plotting. 
For a comparison see http://pyqtgraph.org/ (actually not that biased)

The same API works for plotting a measured dataset or an old  dataset.

`DataSet` objects are not intended to be instantiated directly, but
rather through the helper functions:
- `load_data` for existing data sets, including the data currently
  being acquired.
- `new_data` to make an empty data set to be populated with new
  measurements or simulation data. `new_data` is called internally by
  `Loop.run()` so is also generally not needed directly.

In [ ]:
loaded_data = qc.load_data('data/2017-02-28/#002_my_first_measurement_14-34-03')
plot = qc.MatPlot(loaded_data.meter_amplitude)

## Example: multiple 2D measurements with live plotting

In [ ]:
loop = qc.Loop(
    c1[-15:15:1], 0.01).loop(
        c0[-15:12:.5], 0.001).each(
            meter.amplitude, # first measurement, at c2=0 -> amplitude_0 bcs it's action 0
            qc.Task(c2.set, 1), # action 1 -> c2.set(1)
            qc.Wait(0.001),
            meter.amplitude, # second measurement, at c2=1 -> amplitude_4 bcs it's action 4
            qc.Task(c2.set, 0)
    )
data = loop.run(name='2D_test')

### Plot with matplotlib 

In [ ]:
data.sync()

In [ ]:
plot = qc.MatPlot(data.meter_amplitude_0, cmap=plt.cm.hot, subplots=(1, 2))
plot[1].add(data.meter_amplitude_3, cmap=plt.cm.hot)
# data2 = loop.with_bg_task(plot.update, plot.save,  0.05).run()

### The equivalent in QtPlot

In [ ]:
loop = qc.Loop(
    c1[-15:15:1], 0.01).loop(
        c0[-15:12:.5], 0.001).each(
            meter.amplitude, # first measurement, at c2=0 -> amplitude_0 bcs it's action 0
            qc.Task(c2.set, 1), # action 1 -> c2.set(1)
            qc.Wait(0.001),
            meter.amplitude, # second measurement, at c2=1 -> amplitude_4 bcs it's action 4
            qc.Task(c2.set, 0)
    )
data = loop.run(name='2D_test')

In [ ]:
data.sync()

plotQ = qc.QtPlot()
plotQ.add(data.meter_amplitude_0, figsize=(1200, 500))
plotQ.add(data.meter_amplitude_3, subplot=2)

### Example same outer loop, different inner loop 

In [ ]:
loop3 = qc.Loop(
    c1[-15:15:1], 0.1).each(
        qc.Task(c0.set, -10),
        qc.Task(c2.set, 0),
        # a 1D measurement
        meter.amplitude,
        # a 2D sweep, .each is actually unnecessary because this is the default measurement
        qc.Loop(
            c0[-15:15:1], 0.001).each(
                meter.amplitude),
        qc.Task(c0.set, -10),
        # a 2D sweep with the same outer but different inner loop
        qc.Loop(
            c2[-10:10:0.2], 0.001).each(
                meter.amplitude),
        AverageGetter(meter.amplitude, c2[-10:10:0.2], 0.001)
)
data = loop3.run(name='TwoD_different_inner_test') 

### several plots updating simultaneously (Currently broken on matplotlib)

In [ ]:
data.sync()

In [ ]:
plotQ = qc.QtPlot()
plotQ.add(data.meter_amplitude_3_0)
plotQ.add(data.meter_amplitude_5_0, cmap='viridis', subplot=2)
plotQ.add(data.meter_avg_amplitude, subplot=3)
# data = loop3.with_bg_task(plotQ.update, plotQ.save).run()

### Example 2D scan and average

 An example of a parameter that returns several values of different dimension
 This produces the last two arrays from data3, but only takes the data once.

In [ ]:
loop4 = qc.Loop(
    c1[-15:15:1], 0.01).each(
        AverageAndRaw(meter.amplitude, c2[-10:10:0.2], 0.001)
)
data4 = loop4.run(name='TwoD_average_test')

In [ ]:
data.sync()

In [ ]:
plotQ = qc.QtPlot()
plotQ.add(data4.meter_amplitude, figsize=(1200, 500), cmap='viridis')
plotQ.add(data4.meter_avg_amplitude, subplot=2)